# Gathering Data from https://corpus.quran.com/ 

In [3]:
import pandas as pd
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.common.by import By


In [4]:
service = webdriver.ChromeService(executable_path = './chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service)

In [5]:
url = 'https://corpus.quran.com/qurandictionary.jsp?q=Axr'
driver.get(url)


In [4]:
title = driver.find_element(By.TAG_NAME, "h2").text
title = title.split(" - ")[1]
title

'أ خ ر'

In [25]:
table_info = []

# Locate all the tables
tables = driver.find_elements(By.CLASS_NAME, "taf")
last_known_h4 = ""
# Iterate over each table
for table in tables:
    
    preceding_element = table.find_element(By.XPATH, "(preceding-sibling::h4 | preceding-sibling::p)[last()]")
    if preceding_element.tag_name == "p":
        precedingh4 = preceding_element.find_element(By.XPATH, "(preceding-sibling::h4)[last()]")
        if precedingh4:
            last_known_h4 = precedingh4.text
            subheading = last_known_h4 + " " + preceding_element.text
        else:
            subheading = last_known_h4 + " "  + preceding_element.text
    # Get the subheading text
    else:
        subheading = preceding_element.text.strip()    
    print("subheading:", subheading)

    # # Print or use the subheading as needed
    # rows = table.find_elements(By.TAG_NAME, "tr")
    # table_data = []
    
    # # Iterate over each row in the table
    # for row in rows:
    #     columns = row.find_elements(By.TAG_NAME, "td")
    #     c1 = columns[0].text.split(" ")
        
    #     values = {
    #         "location": c1[0],
    #         "transliteration": c1[1],
    #         "translation": columns[1].text,
    #         "arabic_verse_part": columns[2].text,  # Change arabic_verse to arabic_verse_part
    #     }

    #     # Find the span with class "auu" in the last column of the row
    #     span = row.find_elements(By.XPATH, ".//td[last()]//span[@class='auu']")
    #     values["arabic_word"] = span[0].text
        
    #     # Append the row_data to table_data list
    #     table_data.append(values)
    
    # Append the table_data along with the subheading to table_info list
    # table_info.append({"subheading": subheading, "data": table_data})


subheading: Nominal (1) Noun
subheading: Nominal (2) Time adverb


In [122]:
#Axr

In [71]:

table_info = []

# Iterate over each table
for table in driver.find_elements(By.CLASS_NAME, "taf"):
    rows = table.find_elements(By.TAG_NAME, "tr")
    table_data = []
    
    # Iterate over each row in the table
    for row in rows:
        
        columns = row.find_elements(By.TAG_NAME, "td")
        c1 = columns[0].text.split(" ")
        
        values = {
        "location": c1[0],
        "transliteration": c1[1],
        "translation": columns[1].text,
        "arabic_verse_part": columns[2].text,
        }

        # Find the span with class "auu" in the last column of the row
        span = row.find_elements(By.XPATH, ".//td[last()]//span[@class='auu']")
        values["arabic_word"] = span[0].text
        
        # Append the row_data to table_data list
        table_data.append(values)
    
    # Append the table_data to table_info list
    table_info.append(table_data)


In [72]:
table_info[0][0]

{'location': '(4:104:8)',
 'transliteration': 'talamūna',
 'translation': 'suffering',
 'arabic_verse': 'وَلَا تَهِنُوا فِي ابْتِغَاءِ الْقَوْمِ إِنْ تَكُونُوا تَأْلَمُونَ',
 'arabic_word': 'تَأْلَمُونَ'}

In [73]:
dfs=[]
# Iterate over subheadings and table_info
for subheading, data in zip(subheadings, table_info):
    # Construct a DataFrame for the current subheading and table data
    df = pd.DataFrame(data)
    # Add "Subheading" column with the current subheading value
    df["subheading"] = subheading
    # Add "Title" column with the title value
    df["title"] = title
    # Reorder columns
    df = df[["title", "subheading", "location", "transliteration", "translation", "arabic_verse_part", "arabic_word"]]
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)
final_df

,Title,Subheading,location,transliteration,translation,arabic_verse,arabic_word
0,أ ل م,Verb (form I) - to suffer,(4:104:8),talamūna,suffering,وَلَا تَهِنُوا فِي ابْتِغَاءِ الْقَوْمِ إِنْ ت...,تَأْلَمُونَ
1,أ ل م,Verb (form I) - to suffer,(4:104:10),yalamūna,are (also) suffering,إِنْ تَكُونُوا تَأْلَمُونَ فَإِنَّهُمْ يَأْلَم...,يَأْلَمُونَ
2,أ ل م,Verb (form I) - to suffer,(4:104:12),talamūna,you are suffering,فَإِنَّهُمْ يَأْلَمُونَ كَمَا تَأْلَمُونَ وَتَ...,تَأْلَمُونَ
3,أ ل م,Adjective - painful,(2:10:9),alīmun,painful,وَلَهُمْ عَذَابٌ أَلِيمٌ بِمَا كَانُوا يَكْذِب...,أَلِيمٌ
4,أ ل م,Adjective - painful,(2:104:12),alīmun,painful,وَقُولُوا انْظُرْنَا وَاسْمَعُوا وَلِلْكَافِرِ...,أَلِيمٌ
...,...,...,...,...,...,...,...
70,أ ل م,Adjective - painful,(67:28:15),alīmin,painful,أَوْ رَحِمَنَا فَمَنْ يُجِيرُ الْكَافِرِينَ مِ...,أَلِيمٍ
71,أ ل م,Adjective - painful,(71:1:14),alīmun,painful,أَنْ أَنْذِرْ قَوْمَكَ مِنْ قَبْلِ أَنْ يَأْتِ...,أَلِيمٌ
72,أ ل م,Adjective - painful,(73:13:5),alīman,painful,وَطَعَامًا ذَا غُصَّةٍ وَعَذَابًا أَلِيمًا,أَلِيمًا
73,أ ل م,Adjective - painful,(76:31:10),alīman,painful,يُدْخِلُ مَنْ يَشَاءُ فِي رَحْمَتِهِ وَالظَّال...,أَلِيمًا


# NOW to write a whole loop!

In [29]:
url = 'https://corpus.quran.com/qurandictionary.jsp'
driver.get(url)
dfs = []

while True:
    title = driver.find_element(By.TAG_NAME, "h2").text
    title = title.split(" - ")[1]    

    table_info = []

    # Locate all the tables
    tables = driver.find_elements(By.CLASS_NAME, "taf")
    
    last_known_h4 = ""

    # Iterate over each table
    for table in tables:
        
        preceding_element = table.find_element(By.XPATH, "(preceding-sibling::h4 | preceding-sibling::p)[last()]")
        if preceding_element.tag_name == "p":
            precedingh4 = preceding_element.find_element(By.XPATH, "(preceding-sibling::h4)[last()]")
            if precedingh4:
                last_known_h4 = precedingh4.text
                subheading = last_known_h4 + " " + preceding_element.text
            else:
                subheading = last_known_h4 + " "  + preceding_element.text
        # Get the subheading text
        else:
            subheading = preceding_element.text.strip()    

        # Print or use the subheading as needed
        rows = table.find_elements(By.TAG_NAME, "tr")
        table_data = []
        
        # Iterate over each row in the table
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            c1 = columns[0].text.split(" ")
            
            values = {
                "location": c1[0],
                "transliteration": c1[1],
                "translation": columns[1].text,
                "arabic_verse_part": columns[2].text,  # Change arabic_verse to arabic_verse_part
            }

            # Find the span with class "auu" in the last column of the row
            span = row.find_elements(By.XPATH, ".//td[last()]//span[@class='auu']")
            values["arabic_word"] = span[0].text
            
            # Append the row_data to table_data list
            table_data.append(values)
        
        # Append the table_data along with the subheading to table_info list
        table_info.append({"subheading": subheading, "data": table_data})


    for item in table_info:
        subheading = item['subheading']
        data = item['data']
        
        # Construct a DataFrame for the current subheading and table data
        df = pd.DataFrame(data)
        
        # Add "Subheading" column with the current subheading value
        df["subheading"] = subheading
        df["title"] = title
        
        # Reorder columns
        df = df[["title", "subheading", "location", "transliteration", "translation", "arabic_verse_part", "arabic_word"]]
        
        # Append the DataFrame to the list
        dfs.append(df)

    try:
        next_link = driver.find_element(By.XPATH, "//a[descendant::img[@src='/images/nav-next.png']]")
        # Click on the <a> tag
        next_link.click()
    except:
        break

In [30]:
final_df = pd.concat(dfs, ignore_index=True)
final_df

,title,subheading,location,transliteration,translation,arabic_verse_part,arabic_word
0,آدَم,Proper noun - Adam,(2:31:2),ādama,Adam,وَعَلَّمَ آدَمَ الْأَسْمَاءَ كُلَّهَا ثُمَّ عَ...,آدَمَ
1,آدَم,Proper noun - Adam,(2:33:2),yāādamu,O Adam,قَالَ يَا آدَمُ أَنْبِئْهُمْ بِأَسْمَائِهِمْ,يَا آدَمُ
2,آدَم,Proper noun - Adam,(2:34:5),liādama,to Adam,وَإِذْ قُلْنَا لِلْمَلَائِكَةِ اسْجُدُوا لِآدَ...,لِآدَمَ
3,آدَم,Proper noun - Adam,(2:35:2),yāādamu,O Adam,وَقُلْنَا يَا آدَمُ اسْكُنْ أَنْتَ وَزَوْجُكَ ...,يَا آدَمُ
4,آدَم,Proper noun - Adam,(2:37:2),ādamu,Adam,فَتَلَقَّىٰ آدَمُ مِنْ رَبِّهِ كَلِمَاتٍ فَتَا...,آدَمُ
...,...,...,...,...,...,...,...
53919,ي و م,Nominal (2) Time adverb,(82:15:2),yawma,(on the) Day,يَصْلَوْنَهَا يَوْمَ الدِّينِ,يَوْمَ
53920,ي و م,Nominal (2) Time adverb,(83:6:1),yawma,(The) Day,يَوْمَ يَقُومُ النَّاسُ لِرَبِّ الْعَالَمِينَ,يَوْمَ
53921,ي و م,Nominal (2) Time adverb,(83:34:1),fal-yawma,So today,فَالْيَوْمَ الَّذِينَ آمَنُوا مِنَ الْكُفَّارِ...,فَالْيَوْمَ
53922,ي و م,Nominal (2) Time adverb,(86:9:1),yawma,(The) Day,يَوْمَ تُبْلَى السَّرَائِرُ,يَوْمَ


In [31]:
final_df.to_csv("quran_dictionary.csv")

# Verb

In [32]:
url = "https://corpus.quran.com/verbs.jsp"
driver.get(url)

In [34]:
df_verb = pd.DataFrame(columns=["verb", "root", "form", "frequency", "translation"])


In [44]:
tbody = driver.find_element(By.CLASS_NAME, "verbTable")


In [ ]:
rows = tbody.find_elements(By.TAG_NAME, "tr")
for row in rows:
        # Find all columns within the row
        columns = row.find_elements(By.TAG_NAME, "td")

        # Extract data from each column
        verb = columns[0].text.strip()
        if "Verb" in verb:
                continue
        root = columns[1].text.strip()
        form = columns[2].text.strip()
        frequency = columns[3].text.strip()
        translation = columns[4].text.strip()

        # Append the data to the DataFrame
        df_verb = df_verb.append({"verb": verb, "root": root, "form": form, "frequency": frequency, "translation": translation}, ignore_index=True)


# Time for loop

In [49]:
url = "https://corpus.quran.com/verbs.jsp"
driver.get(url)


verbdata = []

while True:
    tbody = driver.find_element(By.CLASS_NAME, "verbTable")

    rows = tbody.find_elements(By.TAG_NAME, "tr")
    for row in rows:
            # Find all columns within the row
            columns = row.find_elements(By.TAG_NAME, "td")

            # Extract data from each column
            verb = columns[0].text.strip()
            if "Verb" in verb:
                    continue
            
            root = columns[1].text.strip()
            form = columns[2].text.strip()
            frequency = columns[3].text.strip()
            translation = columns[4].text.strip()
            verbdata.append({
                  "verb": verb,
                  "root": root,
                  "form": form,
                  "frequency": frequency,
                  "translation": translation,
            })

    try:
        next_link = driver.find_element(By.XPATH, "//a[descendant::img[@src='/images/nav-next.png']]")
        # Click on the <a> tag
        next_link.click()
    except:
        break



In [50]:
verbdata[0]

{'verb': 'قَالَ',
 'root': 'ق و ل',
 'form': 'I',
 'frequency': '1618',
 'translation': 'to say'}

In [51]:
len(verbdata)

1475

In [52]:
verb_df = pd.DataFrame(verbdata)

verb_df

,verb,root,form,frequency,translation
0,قَالَ,ق و ل,I,1618,to say
1,كَانَ,ك و ن,I,1358,to be
2,ءَامَنَ,ا م ن,IV,537,to believe
3,عَلِمَ,ع ل م,I,382,to know
4,جَعَلَ,ج ع ل,I,340,to make
...,...,...,...,...,...
1470,أَتَوَكَّؤُا۟,و ك ا,V,1,to lean
1471,يَتَّكِـُٔ,و ك ا,VIII,1,to recline
1472,وَكَزَ,و ك ز,I,1,to strike
1473,يَلُ,و ل ي,I,1,to be close


In [53]:
verb_df.to_csv("quran_verbs.csv")

# Lemmas

In [6]:
url = "https://corpus.quran.com/lemmas.jsp"
driver.get(url)


lemmadata = []

while True:
    tbody = driver.find_element(By.CLASS_NAME, "lemmaTable")

    rows = tbody.find_elements(By.TAG_NAME, "tr")
    for row in rows:
            # Find all columns within the row
            columns = row.find_elements(By.TAG_NAME, "td")

            # Extract data from each column
            lemma = columns[0].text.strip()
            if "Lemma" in lemma:
                    continue
            
            buckwalter = columns[1].text.strip()
            frequency = columns[2].text.strip()
            pos = columns[3].text.strip()

            lemmadata.append({
                  "lemma": lemma,
                  "buckwalter": buckwalter,
                  "frequency": frequency,
                  "part_of_speech": pos,
            })

    try:
        next_link = driver.find_element(By.XPATH, "//a[descendant::img[@src='/images/nav-next.png']]")
        # Click on the <a> tag
        next_link.click()
    except:
        break



In [7]:
lemmadata[0]

{'lemma': 'مِن',
 'buckwalter': 'min',
 'frequency': '3226',
 'part_of_speech': 'Preposition'}

In [8]:
len(lemmadata)

3680

In [9]:
lemma_df = pd.DataFrame(lemmadata)

lemma_df

,lemma,buckwalter,frequency,part_of_speech
0,مِن,min,3226,Preposition
1,ٱللَّه,{ll~ah,2699,Proper noun
2,فِى,fiY,1701,Preposition
3,إِنّ,<in~,1682,Accusative particle
4,عَلَىٰ,EalaY`,1445,Preposition
...,...,...,...,...
3675,ءَان,'aAn,1,Noun
3676,ءَانَآء,'aAnaA^',1,Time adverb
3677,ءَانِف,'aAnif,1,Noun
3678,ءَانِيَة,'aAniyap,1,Noun


In [10]:
lemma_df.to_csv("quran_lemmas.csv")

# grouped lemmas

In [11]:
url = "https://corpus.quran.com/lemmas.jsp?group=1"
driver.get(url)


lemmadata = []

while True:
    tbody = driver.find_element(By.CLASS_NAME, "lemmaTable")

    rows = tbody.find_elements(By.TAG_NAME, "tr")
    for row in rows:
            # Find all columns within the row
            columns = row.find_elements(By.TAG_NAME, "td")

            # Extract data from each column
            lemma = columns[0].text.strip()
            if "Lemma" in lemma:
                    continue
            
            buckwalter = columns[1].text.strip()
            frequency = columns[2].text.strip()
            pos = columns[3].text.strip()

            lemmadata.append({
                  "lemma": lemma,
                  "buckwalter": buckwalter,
                  "frequency": frequency,
                  "part_of_speech": pos,
            })

    try:
        next_link = driver.find_element(By.XPATH, "//a[descendant::img[@src='/images/nav-next.png']]")
        # Click on the <a> tag
        next_link.click()
    except:
        break



In [12]:
lemmadata[0]

{'lemma': 'مِن',
 'buckwalter': 'min',
 'frequency': '3226',
 'part_of_speech': 'Preposition'}

In [13]:
len(lemmadata)

3357

In [14]:
lemma_df = pd.DataFrame(lemmadata)

lemma_df

,lemma,buckwalter,frequency,part_of_speech
0,مِن,min,3226,Preposition
1,ٱللَّه,{ll~ah,2699,Proper noun
2,مَا,maA,2177,"Relative pronoun (1266), Negative particle (70..."
3,فِى,fiY,1701,Preposition
4,لَا,laA,1691,"Negative particle (1364), Prohibition particle..."
...,...,...,...,...
3352,ءَافِلِين,'aAfiliyn,1,Noun
3353,ءَامِرُون,'aAmiruwn,1,Noun
3354,ءَامِنَة,'aAminap,1,Noun
3355,ءَان,'aAn,1,Noun


In [15]:
lemma_df.to_csv("quran_lemmas_grouped.csv")

In [8]:
file_path = 'quranic-corpus-morphology-0.4.txt'
df = pd.read_csv(file_path, sep='\t')

# Save the DataFrame as a CSV file
df.to_csv('quran_morphology.csv')

In [11]:
df.shape

(128219, 4)

In [10]:
df.head()

,LOCATION,FORM,TAG,FEATURES
0,(1:1:1:1),bi,P,PREFIX|bi+
1,(1:1:1:2),somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN
2,(1:1:2:1),{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN
3,(1:1:3:1),{l,DET,PREFIX|Al+
4,(1:1:3:2),r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN
